# Natural Language Detection

In the [Enigma]("https://github.com/weekendproj/enigma") solver project, I used [Index of Coincidence]("https://en.wikipedia.org/wiki/Index_of_coincidence") as the fitness function to assess us getting "closer" to the actual Enigma setting. Higher IoC score implies more "language-like" structure which is useful so we know we've actually deciphered a message without actually reading all the billions of messages.

Considering how well this very simple frequency-based formula works got me thinking how much better we can get with a little more "intelligent" algorithm. This notebook tries a few Machine Learning based approaches.

Data collection was rather simple. I used free English ebooks from [Project Gutenberg]("https://www.gutenberg.org/") and extracted text from all the `<p></p>` tags (limiting to at least 500 characters to make sure we're getting "meaty" texts). These acts as "perfect" texts with zero mutations. For every text, we generate mutations by randomly changing a few letters with random ones from the same alphabet (see `mutate` function below).

In [31]:
import numpy as np

data_file = 'data/data.txt'

charset_len = 26
charset_offset = 65
trunc_len = 200
mutations_per_string = 15


def hist(string):
    h = [0 for x in range(charset_len)]
    for c in string:
        h[c - charset_offset] += 1
    return h


def ioc(string):
    h, n = hist(string), len(string)
    numerator = sum([x*x-1 for x in h])
    denominator = (n * (n-1)) / charset_len
    
    return numerator / denominator


def mutate(string):
    output = []
    no_of_mutations = np.random.randint(1, trunc_len+1, mutations_per_string)
    
    for mut in no_of_mutations:
        str_copy = string.copy()
        mut_locations = np.random.randint(0, trunc_len, mut)
        for loc in mut_locations:
            str_copy[loc] = np.random.randint(0, charset_len) + charset_offset
        output.append((str_copy, mut))
    return output


def bigrams(string):
    bigrams = [0 for w in range(charset_len**2)]
    
    for i in range(1, len(string)):
        pre = string[i-1] - charset_offset
        post = string[i] - charset_offset
        
        bigrams[pre * charset_len + post] += 1

    return bigrams


def record(hist, bigrams, score, pct_mutations):
    return {
        'hist': hist,
        'bigrams': bigrams + hist,
        'score': score,
        'pct_mutations': pct_mutations
    }

In [32]:
%%time

import pandas as pd

df = []
with open(data_file, 'r') as file:
    for line in file.readlines():
        if len(line) < trunc_len:
            continue

        line = [ord(c) for c in line]
        line = line[:trunc_len]
        
        df.append(record(hist(line), bigrams(mut_string), ioc(line), 0.))
        
        for mutation in mutate(line):
            mut_string, muts = mutation
            
            df.append(record(hist(mut_string), bigrams(mut_string), ioc(mut_string), muts / trunc_len))

df = pd.DataFrame(df)
df.head()

CPU times: user 27.6 s, sys: 173 ms, total: 27.7 s
Wall time: 27.7 s


,hist,bigrams,score,pct_mutations
0,"[15, 2, 3, 3, 24, 11, 4, 14, 19, 0, 1, 4, 3, 6...","[1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, ...",2.192362,0.000
1,"[14, 4, 4, 7, 21, 13, 7, 12, 13, 5, 2, 6, 6, 5...","[0, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0, ...",1.377085,0.690
2,"[10, 8, 7, 3, 15, 10, 10, 9, 8, 5, 6, 2, 7, 11...","[1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.237286,0.890
3,"[9, 6, 10, 3, 18, 11, 5, 10, 14, 4, 6, 4, 9, 1...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...",1.279095,0.735
4,"[4, 4, 6, 7, 19, 12, 5, 10, 7, 3, 10, 5, 9, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1.348342,0.870


As seen below, IoC is pretty well correlated with percent mutation labels in our generated dataset (negative sign because they arre in different direction). We will use this correlation coefficient as the metric to assess usefulness of our subsequent metrics as it's important for this metric to be tightly correlated with "randomness" in the string which is expressed by the `pct_mutations` metric in this dataset (we know this as we caused these random mutations).

In [33]:
df.corr()

,score,pct_mutations
score,1.000000,-0.881466
pct_mutations,-0.881466,1.000000


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(np.array(df['hist'].tolist()), df['pct_mutations'].values, test_size=.25, shuffle=True)

## SVR with Frequencies

We will start with just character frequencis (same metric IoC is based on) as our featurres. We will use Support Vector Machines as they are good at expanding feature set using kernel functions like polynomials (IoC for example works with squaring every frequency).

In [35]:
from sklearn.svm import SVR

model = SVR(C=.25)
model.fit(X_train, y_train)

SVR(C=0.25)

In [36]:
model.score(X_val, y_val)

0.8830771906346861

In [37]:
np.corrcoef(model.predict(X_val), y_val)

array([[1.        , 0.94048077],
       [0.94048077, 1.        ]])

With 26 features and ~70k training examples, SVR was painfully slow but results are already much better than IoC. We went from ~0.88 to ~0.94 for the correlation coefficient.

In [42]:
X_train, X_val, y_train, y_val = train_test_split(np.array(df['hist'].tolist()), df['pct_mutations'].values, test_size=.25, shuffle=True)

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam

ffnn_model = Sequential()
ffnn_model.add(Dense(100, input_dim=26, activation='relu'))
for i in range(10):
    ffnn_model.add(Dense(100, activation='relu'))

ffnn_model.add(Dense(1, activation='sigmoid'))

ffnn_model.compile(loss='mse', optimizer=Adam(learning_rate=10**-3))

In [50]:
early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=4)

ffnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=16, callbacks=[early_stopping])

Epoch 1/25
3283/3283 [==============================] - 8s 2ms/step - loss: 0.0126 - val_loss: 0.0113
Epoch 2/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0116 - val_loss: 0.0113
Epoch 3/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0114 - val_loss: 0.0108
Epoch 4/25
3283/3283 [==============================] - 8s 2ms/step - loss: 0.0112 - val_loss: 0.0120
Epoch 5/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0112 - val_loss: 0.0108
Epoch 6/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0111 - val_loss: 0.0109
Epoch 7/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0111 - val_loss: 0.0108
Epoch 8/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0110 - val_loss: 0.0107
Epoch 9/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.0110 - val_loss: 0.0109
Epoch 10/25
3283/3283 [==============================] - 7s 2ms/step - loss: 0.010

In [51]:
np.corrcoef(ffnn_model.predict(X_val).T, y_val)

array([[1.        , 0.94099866],
       [0.94099866, 1.        ]])

In [52]:
residual = np.sum(np.power((ffnn_model.predict(X_val).T - y_val), 2))
total = np.sum(np.power((y_val - np.mean(y_val)), 2))

1 - (residual / total)

0.8846960424423719

In [54]:
X_train, X_val, y_train, y_val = train_test_split(np.array(df['bigrams'].tolist()), df['pct_mutations'].values, test_size=.25, shuffle=True)

In [69]:
%%time

model = SVR(C=.25, kernel='poly', degree=2)
model.fit(X_train[:30000], y_train[:30000])

CPU times: user 1min 44s, sys: 79 ms, total: 1min 44s
Wall time: 1min 44s


SVR(C=0.25, degree=2, kernel='poly')

In [70]:
model.score(X_val, y_val)

0.922329419918742

In [71]:
np.corrcoef(model.predict(X_val), y_val)

array([[1.        , 0.96047595],
       [0.96047595, 1.        ]])

In [72]:
X_train, X_val, y_train, y_val = train_test_split(np.array(df['bigrams'].tolist()), df['pct_mutations'].values, test_size=.25, shuffle=True)

In [96]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam

ffnn_model = Sequential()
ffnn_model.add(Dense(250, input_dim=len(X_train[0]), activation='relu'))
for i in range(20):
    ffnn_model.add(Dense(250, activation='relu'))
    ffnn_model.add(Dropout(.01))

ffnn_model.add(Dense(1, activation='sigmoid'))

ffnn_model.compile(loss='mse', optimizer=Adam(learning_rate=10**-3))

In [97]:
early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=4)

ffnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=16, callbacks=[early_stopping])

Epoch 1/25
3283/3283 [==============================] - 29s 8ms/step - loss: 0.0093 - val_loss: 0.0079
Epoch 2/25
3283/3283 [==============================] - 29s 9ms/step - loss: 0.0074 - val_loss: 0.0067
Epoch 3/25
3283/3283 [==============================] - 29s 9ms/step - loss: 0.0069 - val_loss: 0.0073
Epoch 4/25
3283/3283 [==============================] - 29s 9ms/step - loss: 0.0069 - val_loss: 0.0062
Epoch 5/25
3283/3283 [==============================] - 27s 8ms/step - loss: 0.0066 - val_loss: 0.0064
Epoch 6/25
3283/3283 [==============================] - 27s 8ms/step - loss: 0.0064 - val_loss: 0.0068
Epoch 7/25
3283/3283 [==============================] - 28s 9ms/step - loss: 0.0064 - val_loss: 0.0063
Epoch 8/25
3283/3283 [==============================] - 28s 9ms/step - loss: 0.0061 - val_loss: 0.0074


In [98]:
residual = np.sum(np.power((ffnn_model.predict(X_val).T - y_val), 2))
total = np.sum(np.power((y_val - np.mean(y_val)), 2))

1 - (residual / total)

0.9334016292149889

In [99]:
np.corrcoef(ffnn_model.predict(X_val).T, y_val)

array([[1.        , 0.96649047],
       [0.96649047, 1.        ]])